# Gene Set Enrichment Analysis for AD Risk Genes

**Authors:** Ru Feng and Jenny Empawi

## Motivation

Perform gene set enrichment analysis for AD risk genes identified via xQTL. Highlight the pathway enrichment for novel genes to uncover new insights.

## Implementation

Outline the steps for performing pathway enrichment analysis for the identified AD risk genes.